In [ ]:
import torch
import pandas as pd
from transformers import XLNetModel
import numpy as np
from torch.nn import functional as F
from torch import nn as nn
import math

In [ ]:
class xlnet_model(torch.nn.Module):
    def __init__(self):
        super(xlnet_model,self).__init__()
        self.xlnet = XLNetModel.from_pretrained('XLNet-path')
        for param in self.xlnet.parameters():
            param.requires_grad = False
            
    def forward(self,x1,mask):
        encoder_out = self.xlnet(x1,attention_mask=mask)[0]
        return encoder_out

In [ ]:
class deepCNN(nn.Module):
    def __init__(self):
        super(deepCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(20,128,kernel_size=6,padding='same'),
            nn.ReLU(),
            nn.MaxPool1d(1)
        )
        self.dropout = nn.Dropout(0.5)
        self.flatten = nn.Flatten()

    def forward(self,inputs):
        inputs = inputs.view(inputs.shape[0],-1,1)
        out = self.conv(inputs)
        out = self.dropout(out)
        out = self.flatten(out)
        return out

class GRU_LSTM(nn.Module):
    def __init__(self):
        super(GRU_LSTM, self).__init__()
        self.GRU_layer = nn.GRU(input_size=768, hidden_size=128, batch_first=True)
        self.encoder = nn.LSTM(input_size=128,
                               hidden_size=128,
                               num_layers=2,
                               batch_first=True,
                               dropout=0.5,
                               bidirectional=True)
        self.decoder = nn.Linear(2 * 128, 128)
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.5)

    def forward(self,x):
        x = x.view(x.shape[0],1,-1)
        gru,_ = self.GRU_layer(x)
        outputs, _ = self.encoder(gru)
        out = self.decoder(outputs)
        out = self.flatten(out)
        return out
    
class DeepSmell(nn.Module):
    def __init__(self):
        super(DeepSmell, self).__init__()
        self.xlnet = xlnet_model()
        self.gru_lstm = GRU_LSTM()
        self.w_omega = nn.Parameter(torch.Tensor(
            128 * 2, 128 * 2))
        self.u_omega = nn.Parameter(torch.Tensor(128 * 2, 1))
        

        nn.init.uniform_(self.w_omega, -0.1, 0.1)
        nn.init.uniform_(self.u_omega, -0.1, 0.1)
        
        self.cnn = deepCNN()
        self.dropout = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.q = nn.Linear(256,256)
        self.k = nn.Linear(256,256)
        self.v = nn.Linear(256,256)
        self.fc = nn.Linear(256, 1)

    def forward(self, inputs,mask,x2,labels):
        encoder_out = self.xlnet(inputs,mask)
        encoder_out,_ = torch.max(encoder_out,dim=1)
        out_1 = self.gru_lstm(encoder_out)
        
        out_2 = self.cnn(x2)
        out = torch.cat([out_1,out_2],dim=1)
        out = out.view(out.shape[0],1,-1)
        query, key, value = self.q(out), self.k(out), self.v(out)
        attention = F.softmax(query @ key.transpose(1, 2) / math.sqrt(query.size(2)), dim=-1) @ value
        out = self.flatten(attention)
        out = self.fc(out)
        out = F.sigmoid(out)
        return out